In [1]:
import pandas as pd
import numpy as np


In [2]:
data=pd.read_csv("preprocessed_data_model_B.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             614 non-null    int64  
 1   Gender                 614 non-null    object 
 2   Married                614 non-null    object 
 3   Dependents             614 non-null    int64  
 4   Education              614 non-null    object 
 5   Self_Employed          614 non-null    object 
 6   ApplicantIncome        614 non-null    float64
 7   CoapplicantIncome      614 non-null    float64
 8   LoanAmount             614 non-null    float64
 9   Loan_Amount_Term       614 non-null    float64
 10  Credit_History         614 non-null    float64
 11  Property_Area          614 non-null    object 
 12  Loan_Status            614 non-null    object 
 13  Applicant_log          614 non-null    float64
 14  CoapplicantIncome_log  614 non-null    float64
 15  LoanAm

In [4]:
data

,Unnamed: 0,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Applicant_log,CoapplicantIncome_log,LoanAmount_log,new_income,loan,EMI
0,0,Male,No,0,Graduate,No,5849.0,0.0,146.412162,360.0,1.0,Urban,yes,8.674197,0.000000,4.993232,very high,high,0.406700
1,1,Male,Yes,1,Graduate,No,4583.0,1508.0,128.000000,360.0,1.0,Rural,no,8.430327,7.319202,4.859812,high,medium,0.355556
2,2,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.000000,360.0,1.0,Urban,yes,8.006701,0.000000,4.204693,medium,low,0.183333
3,3,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.000000,360.0,1.0,Urban,yes,7.857094,7.765993,4.795791,low,medium,0.333333
4,4,Male,No,0,Graduate,No,6000.0,0.0,141.000000,360.0,1.0,Urban,yes,8.699681,0.000000,4.955827,very high,high,0.391667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,609,Female,No,0,Graduate,No,2900.0,0.0,71.000000,360.0,1.0,Rural,yes,7.972811,0.000000,4.276666,medium,low,0.197222
610,610,Male,Yes,3,Graduate,No,4106.0,0.0,40.000000,180.0,1.0,Rural,yes,8.320448,0.000000,3.713572,high,low,0.222222
611,611,Male,Yes,1,Graduate,No,8072.0,240.0,253.000000,360.0,1.0,Urban,yes,8.996280,5.484797,5.537334,very high,very high,0.702778
612,612,Male,Yes,2,Graduate,No,7583.0,0.0,187.000000,360.0,1.0,Urban,yes,8.933796,0.000000,5.236442,very high,very high,0.519444


In [5]:
data=data.drop("Unnamed: 0",axis=1)

In [6]:
data["new_income"].unique()

array(['very high', 'high', 'medium', 'low'], dtype=object)

In [7]:
len(data.columns)

18

In [8]:
data.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status',
       'Applicant_log', 'CoapplicantIncome_log', 'LoanAmount_log',
       'new_income', 'loan', 'EMI'],
      dtype='object')

In [9]:
numeric_cols = [
    'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
    'Loan_Amount_Term', 'Credit_History', 'Applicant_log',
    'CoapplicantIncome_log', 'LoanAmount_log', 'EMI'
]

categorical_cols = [
    'Gender', 'Married',  'Education',
    'Self_Employed', 'Property_Area', 
]
ordinal_cols = ['loan', 'new_income', 'Dependents']
ordinal_category=ordinal_category = [
    ['very high', 'high', 'medium', 'low'],  # for 'loan'
    ['very high', 'high', 'medium', 'low'],  # for 'new_income'
    [0, 1, 2, 3]                              # for 'Dependents'
]

In [10]:
data['Loan_Status'] = data['Loan_Status'].map({'yes': 1, 'no': 0})

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder,OrdinalEncoder
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier


# 1. Prepare data
X = data.drop(columns=['Loan_Status'])   # remove actual target
y = data['Loan_Status']     



In [12]:
y.value_counts(normalize=True)

1    0.687296
0    0.312704
Name: Loan_Status, dtype: float64

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)


In [23]:
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_cols),
    ('cat', OrdinalEncoder(), categorical_cols),
    ("ord",OrdinalEncoder(categories=ordinal_category),ordinal_cols )
])

In [69]:

from sklearn.linear_model import LogisticRegression
rf=RandomForestClassifier()
rfe_selector = RFE(estimator=LogisticRegression(max_iter=1000, class_weight='balanced'), n_features_to_select=8
                  )
pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('select', rfe_selector)  # Top 6 features
    
])

In [70]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['ApplicantIncome',
                                                   'CoapplicantIncome',
                                                   'LoanAmount',
                                                   'Loan_Amount_Term',
                                                   'Credit_History',
                                                   'Applicant_log',
                                                   'CoapplicantIncome_log',
                                                   'LoanAmount_log', 'EMI']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['Gender', 'Married',
                                                   'Education', 'Self_Employed',
                                                   'Property_Area']),
                           

In [71]:
selected_mask = pipeline.named_steps['select'].get_support()

all_feature_names = np.array(numeric_cols + categorical_cols + ordinal_cols)

# Apply mask to get selected feature names
selected_features = all_feature_names[selected_mask]
print("Top Selected Features:", selected_features)


Top Selected Features: ['LoanAmount' 'Loan_Amount_Term' 'Applicant_log' 'CoapplicantIncome_log'
 'EMI' 'Married' 'Education' 'Property_Area']


In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'RandomForest': RandomForestClassifier(),
    'NaiveBayes': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

results = {}

for name, model in models.items():
    pipeline = Pipeline([
        ('preprocess', preprocessor),
        ('select', rfe_selector),
        ('model', model)
    ])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    
    results[name] = acc
    print(f"{name} Accuracy: {acc:.4f}")

LogisticRegression Accuracy: 0.6341
RandomForest Accuracy: 0.6748
NaiveBayes Accuracy: 0.6098
KNN Accuracy: 0.6179
XGBoost Accuracy: 0.7073


In [68]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# Example: RandomForest Grid
pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('select', rfe_selector),
    ('model', RandomForestClassifier(class_weight='balanced'))
])

param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [4, 8, None],
    'model__min_samples_split': [2, 5]
} 

grid = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(5), scoring='roc_auc')
grid.fit(X_train, y_train)

print("Best Params:", grid.best_params_)
print("Best ROC AUC (CV):", grid.best_score_)

Best Params: {'model__max_depth': None, 'model__min_samples_split': 5, 'model__n_estimators': 100}
Best ROC AUC (CV): 0.5309575462346711


In [73]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# Example: RandomForest Grid
pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('select', rfe_selector),
    ('model', XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight=2.2))
])

param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [4, 8],
    'model__learning_rate': [0.01, 0.1]
}

grid = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(5), scoring='roc_auc')
grid.fit(X_train, y_train)

print("Best Params:", grid.best_params_)
print("Best ROC AUC (CV):", grid.best_score_)

Best Params: {'model__learning_rate': 0.01, 'model__max_depth': 4, 'model__n_estimators': 100}
Best ROC AUC (CV): 0.5410865737725038


In [74]:
grid.best_estimator_

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['ApplicantIncome',
                                                   'CoapplicantIncome',
                                                   'LoanAmount',
                                                   'Loan_Amount_Term',
                                                   'Credit_History',
                                                   'Applicant_log',
                                                   'CoapplicantIncome_log',
                                                   'LoanAmount_log', 'EMI']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['Gender', 'Married',
                                                   'Education', 'Self_Employed',
                                                   'Property_Area']),
                           

In [75]:
best_pipeline = grid.best_estimator_
selected_mask = best_pipeline.named_steps['select'].get_support()

all_feature_names = np.array(numeric_cols + categorical_cols + ordinal_cols)

# Apply mask to get selected feature names
selected_features = all_feature_names[selected_mask]
print("Top Selected Features:", selected_features)


Top Selected Features: ['LoanAmount' 'Loan_Amount_Term' 'Applicant_log' 'CoapplicantIncome_log'
 'EMI' 'Married' 'Education' 'Property_Area']


In [76]:
import joblib
joblib.dump(best_pipeline,'loan_default_pipeline.pkl')

['loan_default_pipeline.pkl']

In [78]:
cat_encoder = pipeline.named_steps['preprocess'].named_transformers_['cat']
print(cat_encoder.categories_)

[array(['Female', 'Male'], dtype=object), array(['No', 'Yes'], dtype=object), array(['Graduate', 'Not Graduate'], dtype=object), array(['No', 'Yes'], dtype=object), array(['Rural', 'Semiurban', 'Urban'], dtype=object)]


In [80]:
import pandas as pd


# Load the trained pipeline
pipeline = joblib.load('loan_default_pipeline.pkl')

# All columns required by pipeline (same as training data)
all_columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
               'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term',
               'Credit_History', 'Property_Area', 'Applicant_log', 'CoapplicantIncome_log',
               'LoanAmount_log', 'new_income', 'loan', 'EMI']

# Default values (just placeholders — replace with training medians/modes later)
defaults = {
    'Gender': 'Male',
    'Married': 'Yes',
    'Dependents': 0,
    'Education': 'Graduate',
    'Self_Employed': 'No',
    'ApplicantIncome': 4000,
    'CoapplicantIncome': 1000,
    'LoanAmount': 100,
    'Loan_Amount_Term': 360.0,
    'Credit_History': 1.0,
    'Property_Area': 'Urban',
    'Applicant_log': 8.5,
    'CoapplicantIncome_log': 6.9,
    'LoanAmount_log': 4.5,
    'new_income': 'medium',
    'loan': 'low',
    'EMI': 1500
}

# User input only partial (e.g., top 6 important ones)
user_input = {
    'LoanAmount': 120,
    'Loan_Amount_Term': 360.0,
    'Applicant_log': 8.7,
    'CoapplicantIncome_log': 6.2,
    'EMI': 2000,
    'Married': 'No',                 
    'Education': 'Graduate',         
    'Property_Area': 'Semiurban'     
}

# Merge with defaults to get full input row
input_row = defaults.copy()
input_row.update(user_input)

# Convert to DataFrame
new_data = pd.DataFrame([input_row])

# Predict
prediction = pipeline.predict(new_data)
print("Loan Default Prediction:", prediction)

Loan Default Prediction: [1]
